In [6]:
from pyiron_base import GenericJob, DataContainer, state, Executable
import os
from pyiron_base import Project

In [32]:
class ParaprobeRanger(GenericJob):
    def __init__(self, project, job_name):
        super().__init__(project, job_name) 
        self.input = DataContainer(table_name="input")
        self.input.input_path = None
        self._executable = None
        self._executable_activate()
    
    def _executable_activate(self, enforce = False):
        if self._executable is None or enforce:
            self._executable = Executable(
                codename='paraprobe-ranger',
                module='paraprobe-ranger',
                path_binary_codes=state.settings.resource_paths
            )
            
    def write_input(self): 
        pass

    def collect_output(self):
        pass

    def to_hdf(self, hdf=None, group_name=None): 
        super().to_hdf(
            hdf=hdf,
            group_name=group_name
        )
        with self.project_hdf5.open("input") as h5in:
            self.input.to_hdf(h5in)

    def from_hdf(self, hdf=None, group_name=None): 
        super().from_hdf(
            hdf=hdf,
            group_name=group_name
        )
        with self.project_hdf5.open("input") as h5in:
            self.input.from_hdf(h5in)

In [33]:
pr = Project('test')
job = pr.create_job(job_type=ParaprobeRanger, job_name="t1")

In [34]:
job._executable.mpi = True

In [35]:
job._executable

'/home/menon/pyiron/resources/paraprobe-ranger/bin/run_paraprobe-ranger_1.0.3_mpi.sh'